# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ralte\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Read your api key environment variable
api_key = os.getenv("NEWS_API_KEY")

In [3]:
# Create a newsapi client
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key="7286e12a919b465aa48622344d8ac3c2")

In [4]:
# Fetch the Bitcoin news articles
bitcoin_news_en = newsapi.get_everything(
    q="Bitcoin",
    language="en"
)
bitcoin_news_en["totalResults"]

def create_df(news, language):
    articles = []
    for article in news:
        try:
            title = article["title"]
            description = article["description"]
            text = article["content"]
            date = article["publishedAt"][:10]

            articles.append({
                "title": title,
                "description": description,
                "text": text,
                "date": date,
                "language": language
            })
        except AttributeError:
            pass

    return pd.DataFrame(articles)

bitcoin_en_df = create_df(bitcoin_news_en["articles"], "en")
bitcoin_en_df.head()

,title,description,text,date,language
0,A fake press release claiming Kroger accepts c...,A crypto hoax claimed Kroger is accepting Bitc...,A similar hoax earlier this year tied Walmart ...,2021-11-05,en
1,Bitcoin Miners Are Gobbling Up U.S. Energy,There’s a big new presence slurping up power f...,Theres a big new presence slurping up power fr...,2021-10-28,en
2,Roughly One-Third of Bitcoin Is Controlled by ...,"For all the talk of democratizing finance, the...","For all the talk of democratizing finance, the...",2021-10-26,en
3,AMC Begins Accepting Crypto,"Bitcoin, Ethereum, Bitcoin Cash and Litecoin a...","In keeping with a previous announcement, AMC t...",2021-11-12,en
4,Bitcoin edges off all-time high but momentum f...,Bitcoin fell slightly in Asian hours on Thursd...,Representation of cryptocurrency Bitcoin is pl...,2021-10-21,en


In [5]:
# Fetch the Ethereum news articles
ethereum_news_en = newsapi.get_everything(
    q="Ethereum",
    language="en"
)
ethereum_news_en["totalResults"]

def create_df(news, language):
    articles = []
    for article in news:
        try:
            title = article["title"]
            description = article["description"]
            text = article["content"]
            date = article["publishedAt"][:10]

            articles.append({
                "title": title,
                "description": description,
                "text": text,
                "date": date,
                "language": language
            })
        except AttributeError:
            pass

    return pd.DataFrame(articles)

ethereum_en_df = create_df(ethereum_news_en["articles"], "en")
ethereum_en_df.head()

,title,description,text,date,language
0,The NFT Bay asks if you would steal all the JPEGs,The NFT Bay is an incredible piece of performa...,"If you claim ownership, someone will probably ...",2021-11-19,en
1,AMC Begins Accepting Crypto,"Bitcoin, Ethereum, Bitcoin Cash and Litecoin a...","In keeping with a previous announcement, AMC t...",2021-11-12,en
2,JPMorgan says ethereum is a better bet than bi...,Ethereum is the more resilient cryptocurrency ...,Ethereum and bitcoin are the two biggest crypt...,2021-11-05,en
3,A meme coin named after Elon Musk rode the wav...,The surge in October pushed the ethereum-based...,Elon Musk\r\npicture alliance / Getty Images\r...,2021-11-01,en
4,Obscure altcoin mana spikes 400% as Facebook's...,"The price of Decentraland, whose ticker is man...",Cryptocurrency and business continuity line im...,2021-11-01,en


In [6]:
nltk.download("vader_lexicon")
analyzer = SentimentIntensityAnalyzer()

# Create the Bitcoin sentiment scores DataFrame
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result

# Sentiment scores dictionaries
text_sent = {
    "text_compound": [],
    "text_pos": [],
    "text_neu": [],
    "text_neg": [],
    "text_sent": [],
}

# Get sentiment for the text
for index, row in bitcoin_en_df.iterrows():
    try:
        # Sentiment scoring with VADER
        text_sentiment = analyzer.polarity_scores(row["text"])
        text_sent["text_compound"].append(text_sentiment["compound"])
        text_sent["text_pos"].append(text_sentiment["pos"])
        text_sent["text_neu"].append(text_sentiment["neu"])
        text_sent["text_neg"].append(text_sentiment["neg"])
        text_sent["text_sent"].append(get_sentiment(text_sentiment["compound"]))
    except AttributeError:
        pass

# Attaching sentiment columns to the News DataFrame
text_sentiment_df = pd.DataFrame(text_sent)
bitcoin_news_en_df = bitcoin_en_df.join(text_sentiment_df)

bitcoin_news_en_df.head()

# Describe the Bitcoin Sentiment
bitcoin_news_en_df.describe()

bitcoin_news_en_df.head()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ralte\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,title,description,text,date,language,text_compound,text_pos,text_neu,text_neg,text_sent
0,A fake press release claiming Kroger accepts c...,A crypto hoax claimed Kroger is accepting Bitc...,A similar hoax earlier this year tied Walmart ...,2021-11-05,en,-0.2732,0.000,0.937,0.063,-1
1,Bitcoin Miners Are Gobbling Up U.S. Energy,There’s a big new presence slurping up power f...,Theres a big new presence slurping up power fr...,2021-10-28,en,0.3612,0.096,0.904,0.000,1
2,Roughly One-Third of Bitcoin Is Controlled by ...,"For all the talk of democratizing finance, the...","For all the talk of democratizing finance, the...",2021-10-26,en,0.0000,0.000,1.000,0.000,0
3,AMC Begins Accepting Crypto,"Bitcoin, Ethereum, Bitcoin Cash and Litecoin a...","In keeping with a previous announcement, AMC t...",2021-11-12,en,0.5719,0.153,0.847,0.000,1
4,Bitcoin edges off all-time high but momentum f...,Bitcoin fell slightly in Asian hours on Thursd...,Representation of cryptocurrency Bitcoin is pl...,2021-10-21,en,0.0000,0.000,1.000,0.000,0


In [7]:
# Create the Ethereum sentiment scores DataFrame

nltk.download("vader_lexicon")
analyzer = SentimentIntensityAnalyzer()

# Create the Ethereum sentiment scores DataFrame
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result

# Sentiment scores dictionaries

text_sent = {
    "text_compound": [],
    "text_pos": [],
    "text_neu": [],
    "text_neg": [],
    "text_sent": [],
}

# Get sentiment for the text
for index, row in ethereum_en_df.iterrows():
    try:
        # Sentiment scoring with VADER
        text_sentiment = analyzer.polarity_scores(row["text"])
        text_sent["text_compound"].append(text_sentiment["compound"])
        text_sent["text_pos"].append(text_sentiment["pos"])
        text_sent["text_neu"].append(text_sentiment["neu"])
        text_sent["text_neg"].append(text_sentiment["neg"])
        text_sent["text_sent"].append(get_sentiment(text_sentiment["compound"]))
    except AttributeError:
        pass

# Attaching sentiment columns to the News DataFrame
text_sentiment_df = pd.DataFrame(text_sent)
ethereum_news_en_df = ethereum_en_df.join(text_sentiment_df)

ethereum_news_en_df.head()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ralte\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,text_compound,text_pos,text_neu,text_neg,text_sent
count,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.307340,0.077450,0.916300,0.006200,0.550000
std,0.340277,0.078558,0.077373,0.019083,0.604805
min,-0.318200,0.000000,0.779000,0.000000,-1.000000
25%,0.000000,0.000000,0.862750,0.000000,0.000000
50%,0.381550,0.074000,0.915500,0.000000,1.000000
75%,0.549700,0.111750,1.000000,0.000000,1.000000
max,0.822500,0.221000,1.000000,0.062000,1.000000


In [9]:
# Describe the Bitcoin Sentiment
bitcoin_news_en_df.describe()

bitcoin_news_en_df.head()

,title,description,text,date,language,text_compound,text_pos,text_neu,text_neg,text_sent
0,A fake press release claiming Kroger accepts c...,A crypto hoax claimed Kroger is accepting Bitc...,A similar hoax earlier this year tied Walmart ...,2021-11-05,en,-0.2732,0.000,0.937,0.063,-1
1,Bitcoin Miners Are Gobbling Up U.S. Energy,There’s a big new presence slurping up power f...,Theres a big new presence slurping up power fr...,2021-10-28,en,0.3612,0.096,0.904,0.000,1
2,Roughly One-Third of Bitcoin Is Controlled by ...,"For all the talk of democratizing finance, the...","For all the talk of democratizing finance, the...",2021-10-26,en,0.0000,0.000,1.000,0.000,0
3,AMC Begins Accepting Crypto,"Bitcoin, Ethereum, Bitcoin Cash and Litecoin a...","In keeping with a previous announcement, AMC t...",2021-11-12,en,0.5719,0.153,0.847,0.000,1
4,Bitcoin edges off all-time high but momentum f...,Bitcoin fell slightly in Asian hours on Thursd...,Representation of cryptocurrency Bitcoin is pl...,2021-10-21,en,0.0000,0.000,1.000,0.000,0


In [8]:
# Describe the Ethereum Sentiment
ethereum_news_en_df.describe()

,text_compound,text_pos,text_neu,text_neg,text_sent
count,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.307340,0.077450,0.916300,0.006200,0.550000
std,0.340277,0.078558,0.077373,0.019083,0.604805
min,-0.318200,0.000000,0.779000,0.000000,-1.000000
25%,0.000000,0.000000,0.862750,0.000000,0.000000
50%,0.381550,0.074000,0.915500,0.000000,1.000000
75%,0.549700,0.111750,1.000000,0.000000,1.000000
max,0.822500,0.221000,1.000000,0.062000,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: 

Q: Which coin had the highest compound score?

A: 

Q. Which coin had the highest positive score?

A: 

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [10]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [10]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
sw = set(stopwords.words('english'))

# Expand the default stopwords list if necessary
sw_addon = {'said', 'asked', 'told'}

NameError: name 'WordNetLemmatizer' is not defined

In [12]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text

   
    # Create a tokenized list of the words
    
    
    # Lemmatize words into root words

   
    # Convert the words to lowercase
    
    
    # Remove the stop words
    
    
    return tokens

In [13]:
# Create a new tokens column for Bitcoin
# YOUR CODE HERE!

In [14]:
# Create a new tokens column for Ethereum
# YOUR CODE HERE!

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [15]:
from collections import Counter
from nltk import ngrams

In [16]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [17]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [20]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [21]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [22]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [29]:
# Render the visualization
# YOUR CODE HERE!

In [30]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [33]:
# Render the visualization
# YOUR CODE HERE!

In [34]:
# List all Entities
# YOUR CODE HERE!

---